## Multi-class classification using ELECTRA

In [12]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
pip install accelerate -U

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from transformers import ElectraTokenizer, ElectraForSequenceClassification, TrainingArguments, Trainer
import torch
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path
# [5e-05, 0.135161130336292, 5, 13, 13]

def main_model(file_name, ext, type):

    path_type = "Balanced" if type == 1 else "Unbalanced"

    current_file_path = Path(__file__).parent
    path_to_project = current_file_path.parents[1]

    df = pd.read_excel(f"{path_to_project}/Data/Datasets/{path_type}/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/ELECTRA/Output/{path_type}/{file_name}"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    # Select the text and label columns
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)
    X = df['review'].values
    y = df['label'].values

    X_train_CV, X_test_full, y_train_CV, y_test_full = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


    # Encode the labels to a numeric format
    label_encoder = LabelEncoder()
    y_train_CV_encoded = label_encoder.fit_transform(y_train_CV)
    y_test_full_encoded = label_encoder.transform(y_test_full)

    # Initialize the tokenizer for RoBERTa
    tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

    # Tokenization function
    def tokenize_function(texts):
        return tokenizer(texts, padding="max_length", truncation=True, max_length=128)

    loss_logging_callback = LossLoggingCallback()

    # Stratified K-Fold Cross-Validation
    n_splits = 5
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Variables to accumulate scores
    best_accuracy = 0
    best_model = None
    accuracy_scores = []
    metrics_df = pd.DataFrame()


    for fold, (train_index, val_index) in enumerate(kf.split(X_train_CV, y_train_CV_encoded)):
        print(f"Fold {fold+1}/{n_splits}")
        start_time = time.time()
        # Split the data
        X_train, X_val = X_train_CV[train_index], X_train_CV[val_index]
        y_train, y_val = y_train_CV_encoded[train_index], y_train_CV_encoded[val_index]


        # Tokenize the data
        train_encodings = tokenize_function(X_train.tolist())
        val_encodings = tokenize_function(X_val.tolist())

        # Create dataset objects
        train_dataset = ReviewDataset(train_encodings, y_train)
        val_dataset = ReviewDataset(val_encodings, y_val)

        # Initialize the model for each fold
        model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=len(label_encoder.classes_))

        # Define training arguments for each fold, adjust hyperparameters as needed
        training_args = TrainingArguments(
            output_dir=f"{dump_dir}/res",
            num_train_epochs=5,
            per_device_train_batch_size=13,
            per_device_eval_batch_size=13,
            warmup_steps=500,
            weight_decay=0.135161130336292,
            logging_dir=f"{dump_dir}/logs",
            logging_strategy="epoch",
            evaluation_strategy="epoch",
            learning_rate=5e-05,
            max_grad_norm=1.0,
            load_best_model_at_end=True,
            metric_for_best_model="accuracy",
            save_strategy="epoch",
            save_total_limit=2,
            lr_scheduler_type='linear'
        )

        # Initialize Trainer
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=lambda p: {"accuracy": accuracy_score(p.predictions.argmax(-1), p.label_ids)},
            callbacks=[loss_logging_callback]
        )

        # Train
        trainer.train()

        loss_logging_callback.save_logs_to_excel(f"{results_dir}/fold_loss.xlsx")

        # Evaluate
        results = trainer.evaluate()
        accuracy_scores.append(results['eval_accuracy'])

        if results['eval_accuracy'] > best_accuracy:
            best_accuracy = results['eval_accuracy']
            best_model = model  # Assign the best model

        # Get predictions and true labels
        predictions = trainer.predict(val_dataset)
        pred_labels = np.argmax(predictions.predictions, axis=-1)
        true_labels = y_val

        # Calculate accuracy
        accuracy = accuracy_score(true_labels, pred_labels)
        label_names = label_encoder.inverse_transform(range(len(label_encoder.classes_)))

        # Calculate precision, recall, and F1-score
        report_dict = classification_report(true_labels, pred_labels, output_dict=True, zero_division=0, target_names=label_names)
        # avg_metrics = report_dict['weighted avg']  # Use 'macro avg' or 'weighted avg' based on your preference
        end_time = time.time()
        # Append the metrics for this fold to the DataFrame
        metrics_df = metrics_df.append({
            ('Fold', ''): fold + 1,
            ('Accuracy', ''): accuracy,
            ('Train Time', ''): str(end_time - start_time)+" s",
            ('Bug Report', 'P'): report_dict['bug report']['precision'],
            ('Bug Report', 'R'): report_dict['bug report']['recall'],
            ('Bug Report', 'F1'): report_dict['bug report']['f1-score'],
            ('Feature Request', 'P'): report_dict['feature request']['precision'],
            ('Feature Request', 'R'): report_dict['feature request']['recall'],
            ('Feature Request', 'F1'): report_dict['feature request']['f1-score'],
            ('Rating', 'P'): report_dict['rating']['precision'],
            ('Rating', 'R'): report_dict['rating']['recall'],
            ('Rating', 'F1'): report_dict['rating']['f1-score'],
            ('User Experience', 'P'): report_dict['user experience']['precision'],
            ('User Experience', 'R'): report_dict['user experience']['recall'],
            ('User Experience', 'F1'): report_dict['user experience']['f1-score']
        }, ignore_index=True)

    # Save the DataFrame to a CSV file after completing all folds
    metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in metrics_df.columns])
    metrics_df.to_excel(f"{results_dir}/fold_metrics.xlsx", index=True)

    # Evaluate the best model on the test set
    test_encodings = tokenize_function(X_test_full.tolist())
    test_dataset = ReviewDataset(test_encodings, y_test_full_encoded)
    test_trainer = Trainer(model=best_model)
    test_results = test_trainer.predict(test_dataset)
    test_predictions = np.argmax(test_results.predictions, axis=-1)
    test_accuracy = accuracy_score(y_test_full_encoded, test_predictions)

    label_names_full = label_encoder.inverse_transform(range(len(label_encoder.classes_)))

    # Calculate precision, recall, and F1-score
    report_dict_full = classification_report(y_test_full_encoded, test_predictions, output_dict=True, zero_division=0, target_names=label_names_full)
    print(report_dict_full)
    full_metrics_df = pd.DataFrame()

    full_metrics_df = full_metrics_df.append({
            ('Accuracy', ''): test_accuracy,
            ('Bug Report', 'P'): report_dict_full['bug report']['precision'],
            ('Bug Report', 'R'): report_dict_full['bug report']['recall'],
            ('Bug Report', 'F1'): report_dict_full['bug report']['f1-score'],
            ('Feature Request', 'P'): report_dict_full['feature request']['precision'],
            ('Feature Request', 'R'): report_dict_full['feature request']['recall'],
            ('Feature Request', 'F1'): report_dict_full['feature request']['f1-score'],
            ('Rating', 'P'): report_dict_full['rating']['precision'],
            ('Rating', 'R'): report_dict_full['rating']['recall'],
            ('Rating', 'F1'): report_dict_full['rating']['f1-score'],
            ('User Experience', 'P'): report_dict_full['user experience']['precision'],
            ('User Experience', 'R'): report_dict_full['user experience']['recall'],
            ('User Experience', 'F1'): report_dict_full['user experience']['f1-score']
        }, ignore_index=True)

    full_metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in full_metrics_df.columns])
    full_metrics_df.to_excel(f"{results_dir}/metrics_results_full_test.xlsx", index=True)

    print(f"Test Accuracy: {test_accuracy}")

    # Generate and print the classification report
    print(classification_report(y_test_full_encoded, test_predictions, target_names=label_encoder.classes_, zero_division=0))

    shutil.rmtree(dump_dir)

# Custom dataset class
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

class LossLoggingCallback(TrainerCallback):
    """A custom callback to log training and validation loss."""
    def __init__(self):
        super().__init__()
        self.log_history = []
        self.log_train_loss_history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        # This method captures both training and evaluation logs, so it's more general than on_epoch_end
        if logs is not None:
            # Capture both training and evaluation steps
            if 'loss' in logs:  # Indicates a training step
                self.log_train_loss_history.append({
                    'epoch': state.epoch,
                    'training_loss': logs.get('loss'),
                })
            elif 'eval_loss' in logs:  # Indicates an evaluation step
                # Make sure to capture the last training loss as well
                last_training_loss = self.log_train_loss_history[-1]['training_loss'] if self.log_train_loss_history else None
                self.log_history.append({
                    'epoch': state.epoch,
                    'training_loss': last_training_loss,  # Include last known training loss for reference
                    'validation_loss': logs.get('eval_loss'),
                    'eval_runtime':logs.get('eval_runtime')
                })

    def save_logs_to_excel(self, file_name):
        """Save the recorded logs to a Excel file."""
        pd.DataFrame(self.log_history).to_excel(file_name, index=False)

__file__ = "/content/drive/MyDrive/FinalProject/Models/ELECTRA/ELECTRA.ipynb"
current_file_path = Path(__file__).parent
path_to_project = current_file_path.parents[1]

directory_path_multi = path_to_project / 'Data' / 'Datasets' / 'Balanced'

files_multi = [(file.name, file.stat().st_size)
               for file in directory_path_multi.iterdir()
               if file.is_file() and not file.name.startswith('.')]

files_multi.sort(key=lambda x: x[1])





Running each dataset on the model separately due to storage constraints

In [ ]:
print(f"Now doing: {files_multi[1][0].split('.')[0]}")
main_model(files_multi[1][0].split('.')[0], files_multi[1][0].split('.')[1], 1)
print(f"Now doing: {files_multi[2][0].split('.')[0]}")
main_model(files_multi[2][0].split('.')[0], files_multi[2][0].split('.')[1], 1)

Now doing: dataset_balanced_4000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.061200,0.721894,0.746875
2,0.635800,0.663521,0.789062
3,0.352700,0.539904,0.853906
4,0.174700,0.578044,0.878125
5,0.090600,0.547317,0.893750


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Balanced/dataset_balanced_8000/Dump/res/checkpoint-1970 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-5-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9082125603864735, 'recall': 0.94, 'f1-score': 0.9238329238329238, 'support': 400}, 'feature request': {'precision': 0.905, 'recall': 0.905, 'f1-score': 0.905, 'support': 400}, 'rating': {'precision': 0.9093264248704663, 'recall': 0.8775, 'f1-score': 0.8931297709923663, 'support': 400}, 'user experience': {'precision': 0.8575, 'recall': 0.8575, 'f1-score': 0.8575, 'support': 400}, 'accuracy': 0.895, 'macro avg': {'precision': 0.8950097463142349, 'recall': 0.895, 'f1-score': 0.8948656737063225, 'support': 1600}, 'weighted avg': {'precision': 0.8950097463142349, 'recall': 0.895, 'f1-score': 0.8948656737063225, 'support': 1600}}
Test Accuracy: 0.895
                 precision    recall  f1-score   support

     bug report       0.91      0.94      0.92       400
feature request       0.91      0.91      0.91       400
         rating       0.91      0.88      0.89       400
user experience       0.86      0.86      0.86       400

       accuracy             

<ipython-input-5-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [ ]:
print(f"Now doing: {files_multi[3][0].split('.')[0]}")
main_model(files_multi[3][0].split('.')[0], files_multi[3][0].split('.')[1], 1)
print(f"Now doing: {files_multi[4][0].split('.')[0]}")
main_model(files_multi[4][0].split('.')[0], files_multi[4][0].split('.')[1], 1)

Now doing: dataset_gpt_balanced_4000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.362700,1.165973,0.450000
2,0.689700,0.329568,0.914062
3,0.309000,0.242556,0.942187
4,0.234000,0.271135,0.939063
5,0.192700,0.186608,0.953125


<ipython-input-5-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.918900,0.256047,0.935887
2,0.173300,0.187253,0.967162
3,0.144000,0.125035,0.974980
4,0.112000,0.140384,0.974980
5,0.091800,0.132801,0.972635


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Balanced/dataset_gpt_balanced_8000/Dump/res/checkpoint-1970 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-5-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9897172236503856, 'recall': 0.9625, 'f1-score': 0.9759188846641318, 'support': 400}, 'feature request': {'precision': 0.9974619289340102, 'recall': 0.9825, 'f1-score': 0.9899244332493703, 'support': 400}, 'rating': {'precision': 0.9775561097256857, 'recall': 0.98, 'f1-score': 0.978776529338327, 'support': 400}, 'user experience': {'precision': 0.9543269230769231, 'recall': 0.9925, 'f1-score': 0.9730392156862746, 'support': 400}, 'accuracy': 0.979375, 'macro avg': {'precision': 0.9797655463467512, 'recall': 0.979375, 'f1-score': 0.9794147657345259, 'support': 1600}, 'weighted avg': {'precision': 0.9797655463467511, 'recall': 0.979375, 'f1-score': 0.9794147657345259, 'support': 1600}}
Test Accuracy: 0.979375
                 precision    recall  f1-score   support

     bug report       0.99      0.96      0.98       400
feature request       1.00      0.98      0.99       400
         rating       0.98      0.98      0.98       400
user experience       0.

<ipython-input-5-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [ ]:
print(f"Now doing: {files_multi[5][0].split('.')[0]}")
main_model(files_multi[5][0].split('.')[0], files_multi[5][0].split('.')[1], 1)

Now doing: dataset_gpt_balanced_20000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.441200,0.083262,0.985000
2,0.083400,0.065835,0.987187
3,0.062100,0.097049,0.983750
4,0.053900,0.063919,0.987812
5,0.046200,0.067694,0.988750


<ipython-input-5-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.419800,0.091060,0.982495
2,0.075000,0.082893,0.984058
3,0.061400,0.073411,0.988121
4,0.053900,0.069624,0.987496
5,0.042300,0.071269,0.986558


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Balanced/dataset_gpt_balanced_20000/Dump/res/checkpoint-4925 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-5-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9989868287740629, 'recall': 0.986, 'f1-score': 0.992450931051837, 'support': 1000}, 'feature request': {'precision': 1.0, 'recall': 0.99, 'f1-score': 0.9949748743718593, 'support': 1000}, 'rating': {'precision': 0.9919759277833501, 'recall': 0.989, 'f1-score': 0.9904857285928894, 'support': 1000}, 'user experience': {'precision': 0.9746588693957114, 'recall': 1.0, 'f1-score': 0.9871668311944718, 'support': 1000}, 'accuracy': 0.99125, 'macro avg': {'precision': 0.9914054064882811, 'recall': 0.99125, 'f1-score': 0.9912695913027643, 'support': 4000}, 'weighted avg': {'precision': 0.9914054064882811, 'recall': 0.99125, 'f1-score': 0.9912695913027643, 'support': 4000}}
Test Accuracy: 0.99125
                 precision    recall  f1-score   support

     bug report       1.00      0.99      0.99      1000
feature request       1.00      0.99      0.99      1000
         rating       0.99      0.99      0.99      1000
user experience       0.97      1.00      0.

<ipython-input-5-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [4]:
print(f"Now doing: {files_multi[6][0].split('.')[0]}")
main_model(files_multi[6][0].split('.')[0], files_multi[6][0].split('.')[1], 1)

Now doing: dataset_gpt_balanced_32000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.278100,0.064412,0.991209
2,0.062000,0.050653,0.992772
3,0.043600,0.046889,0.992967
4,0.040300,0.041409,0.992967
5,0.030300,0.036905,0.993944


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Balanced/dataset_gpt_balanced_32000/Dump/res/checkpoint-7880 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9993706733794839, 'recall': 0.9925, 'f1-score': 0.9959234869865162, 'support': 1600}, 'feature request': {'precision': 1.0, 'recall': 0.9925, 'f1-score': 0.9962358845671266, 'support': 1600}, 'rating': {'precision': 0.9931120851596744, 'recall': 0.99125, 'f1-score': 0.9921801689083515, 'support': 1600}, 'user experience': {'precision': 0.984009840098401, 'recall': 1.0, 'f1-score': 0.9919404835709857, 'support': 1600}, 'accuracy': 0.9940625, 'macro avg': {'precision': 0.9941231496593899, 'recall': 0.9940625000000001, 'f1-score': 0.994070006008245, 'support': 6400}, 'weighted avg': {'precision': 0.9941231496593899, 'recall': 0.9940625, 'f1-score': 0.9940700060082451, 'support': 6400}}
Test Accuracy: 0.9940625
                 precision    recall  f1-score   support

     bug report       1.00      0.99      1.00      1600
feature request       1.00      0.99      1.00      1600
         rating       0.99      0.99      0.99      1600
user experience       0

<ipython-input-3-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [5]:
directory_path_unbalanced = path_to_project / 'Data' / 'Datasets' / 'Unbalanced'

files_unbalanced = [(file.name, file.stat().st_size)
               for file in directory_path_unbalanced.iterdir()
               if file.is_file() and not file.name.startswith('.')]

files_unbalanced.sort(key=lambda x: x[1])

In [6]:
print(f"Now doing: {files_unbalanced[0][0].split('.')[0]}")
main_model(files_unbalanced[0][0].split('.')[0], files_unbalanced[0][0].split('.')[1], 2)
print(f"Now doing: {files_unbalanced[1][0].split('.')[0]}")
main_model(files_unbalanced[1][0].split('.')[0], files_unbalanced[1][0].split('.')[1], 2)

Now doing: dataset_unbalanced_4000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.232000,0.925900,0.679688
2,0.777200,0.588830,0.793750
3,0.539400,0.613534,0.793750
4,0.276700,0.645255,0.831250
5,0.130100,0.685296,0.843750


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.285800,1.005161,0.667188
2,0.500100,0.239002,0.959375
3,0.091600,0.172834,0.973437
4,0.045500,0.112512,0.982812
5,0.026700,0.115472,0.982812


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Unbalanced/dataset_gpt_unbalanced_4000/Dump/res/checkpoint-985 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.98989898989899, 'recall': 0.98, 'f1-score': 0.9849246231155778, 'support': 100}, 'feature request': {'precision': 0.9932885906040269, 'recall': 0.9866666666666667, 'f1-score': 0.9899665551839465, 'support': 150}, 'rating': {'precision': 0.9959514170040485, 'recall': 0.984, 'f1-score': 0.9899396378269618, 'support': 250}, 'user experience': {'precision': 0.980327868852459, 'recall': 0.9966666666666667, 'f1-score': 0.9884297520661156, 'support': 300}, 'accuracy': 0.98875, 'macro avg': {'precision': 0.9898667165898811, 'recall': 0.9868333333333333, 'f1-score': 0.9883151420481505, 'support': 800}, 'weighted avg': {'precision': 0.9888367531090662, 'recall': 0.98875, 'f1-score': 0.9887516008321562, 'support': 800}}
Test Accuracy: 0.98875
                 precision    recall  f1-score   support

     bug report       0.99      0.98      0.98       100
feature request       0.99      0.99      0.99       150
         rating       1.00      0.98      0.99       25

<ipython-input-3-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [7]:
print(f"Now doing: {files_unbalanced[2][0].split('.')[0]}")
main_model(files_unbalanced[2][0].split('.')[0], files_unbalanced[2][0].split('.')[1], 2)

Now doing: dataset_gpt_unbalanced_8000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.885600,0.192889,0.960156
2,0.100200,0.071206,0.987500
3,0.068200,0.055661,0.991406
4,0.040200,0.045501,0.992188
5,0.032000,0.044189,0.992969


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.960100,0.159222,0.969507
2,0.101000,0.060379,0.991400
3,0.049300,0.047026,0.993745
4,0.035900,0.050152,0.992181
5,0.024000,0.055655,0.991400


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Unbalanced/dataset_gpt_unbalanced_8000/Dump/res/checkpoint-1970 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 0.9849246231155779, 'recall': 0.98, 'f1-score': 0.9824561403508771, 'support': 200}, 'feature request': {'precision': 0.9966555183946488, 'recall': 0.9933333333333333, 'f1-score': 0.9949916527545909, 'support': 300}, 'rating': {'precision': 0.9979838709677419, 'recall': 0.99, 'f1-score': 0.9939759036144579, 'support': 500}, 'user experience': {'precision': 0.9834983498349835, 'recall': 0.9933333333333333, 'f1-score': 0.9883913764510779, 'support': 600}, 'accuracy': 0.990625, 'macro avg': {'precision': 0.9907655905782381, 'recall': 0.9891666666666665, 'f1-score': 0.989953768292751, 'support': 1600}, 'weighted avg': {'precision': 0.9906703284539822, 'recall': 0.990625, 'f1-score': 0.9906321884840176, 'support': 1600}}
Test Accuracy: 0.990625
                 precision    recall  f1-score   support

     bug report       0.98      0.98      0.98       200
feature request       1.00      0.99      0.99       300
         rating       1.00      0.99      0.99   

<ipython-input-3-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


In [8]:
print(f"Now doing: {files_unbalanced[3][0].split('.')[0]}")
main_model(files_unbalanced[3][0].split('.')[0], files_unbalanced[3][0].split('.')[1], 2)

Now doing: dataset_gpt_unbalanced_16000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.497900,0.071143,0.989453
2,0.052800,0.064746,0.991016
3,0.038800,0.064255,0.991406
4,0.031400,0.055914,0.992188
5,0.026400,0.057325,0.992188


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Accuracy
1,0.514900,0.101965,0.981633
2,0.055800,0.050255,0.992966
3,0.045700,0.036748,0.994529
4,0.039200,0.028362,0.994529
5,0.024900,0.049673,0.992575


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Unbalanced/dataset_gpt_unbalanced_16000/Dump/res/checkpoint-3940 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-3-9e6a5c40ded1>:144: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


{'bug report': {'precision': 1.0, 'recall': 0.985, 'f1-score': 0.9924433249370278, 'support': 400}, 'feature request': {'precision': 1.0, 'recall': 0.9966666666666667, 'f1-score': 0.9983305509181971, 'support': 600}, 'rating': {'precision': 0.996, 'recall': 0.996, 'f1-score': 0.996, 'support': 1000}, 'user experience': {'precision': 0.9917218543046358, 'recall': 0.9983333333333333, 'f1-score': 0.9950166112956811, 'support': 1200}, 'accuracy': 0.995625, 'macro avg': {'precision': 0.9969304635761589, 'recall': 0.994, 'f1-score': 0.9954476217877266, 'support': 3200}, 'weighted avg': {'precision': 0.9956456953642384, 'recall': 0.995625, 'f1-score': 0.9956236231501708, 'support': 3200}}
Test Accuracy: 0.995625
                 precision    recall  f1-score   support

     bug report       1.00      0.98      0.99       400
feature request       1.00      1.00      1.00       600
         rating       1.00      1.00      1.00      1000
user experience       0.99      1.00      1.00      1200

<ipython-input-3-9e6a5c40ded1>:181: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_metrics_df = full_metrics_df.append({


## Multi-label classification using ELECTRA

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from transformers import RobertaTokenizer, RobertaForSequenceClassification, TrainingArguments, Trainer
import torch
from torch.utils.data import Dataset
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from transformers import TrainerCallback
import os
import shutil
import re
import time
from pathlib import Path

def multi_main_model(file_name, ext):

    current_file_path = Path(__file__).parent

    path_to_project = current_file_path.parents[1]

    df = pd.read_csv(f"{path_to_project}/Data/Datasets/Multi-label/{file_name}.{ext}")

    results_dir = f"{path_to_project}/Models/ELECTRA/Output/Multi-label/{file_name}"
    dump_dir = results_dir+"/Dump"

    if os.path.isdir(results_dir):
        shutil.rmtree(results_dir)

    os.mkdir(results_dir)
    os.mkdir(dump_dir)

    df = df[df['review'].notna() & (df['review'] != '')]
    df['review'] = df['review'].str.replace('[^\x20-\x7E]', '', regex=True)

    X = df['review'].values
    y = df[['bug report', 'user experience', 'rating', 'feature request']].values

    X_train_CV, X_test_full, y_train_CV, y_test_full = train_test_split(X, y, test_size=0.2, random_state=42)

    tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

    def tokenize_function(examples):
        return tokenizer(examples, padding="max_length", truncation=True, max_length=128)

    loss_logging_callback = LossLoggingCallback()

    # K-Fold Cross-Validation
    n_splits = 5
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

    # Variables to accumulate scores
    best_f1 = 0
    best_model = None
    metrics_df = pd.DataFrame()

    for fold, (train_index, val_index) in enumerate(kf.split(X_train_CV, y_train_CV)):
        print(f"Fold {fold+1}/{n_splits}")
        start_time = time.time()

        X_train, X_val = X_train_CV[train_index], X_train_CV[val_index]
        y_train, y_val = y_train_CV[train_index], y_train_CV[val_index]

        train_encodings = tokenize_function(X_train.tolist())
        val_encodings = tokenize_function(X_val.tolist())

        train_dataset = MultiLabelDataset(train_encodings, y_train)
        val_dataset = MultiLabelDataset(val_encodings, y_val)

        model = ElectraForSequenceClassification.from_pretrained('google/electra-base-discriminator', num_labels=4, problem_type="multi_label_classification")

        training_args = TrainingArguments(
            output_dir=f"{dump_dir}/res",
            num_train_epochs=5,
            per_device_train_batch_size=13,
            per_device_eval_batch_size=13,
            warmup_steps=500,
            weight_decay=0.135161130336292,
            logging_dir=f"{dump_dir}/logs",
            logging_strategy="epoch",
            evaluation_strategy="epoch",
            learning_rate=5e-05,
            max_grad_norm=1.0,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            save_strategy="epoch",
            save_total_limit=2,
            lr_scheduler_type='linear'
        )

        def compute_metrics(p):
            predictions, labels = p
            predictions = torch.sigmoid(torch.tensor(predictions)).numpy()
            threshold = 0.5
            predictions = (predictions > threshold).astype(int)
            precision = precision_score(labels, predictions, average='micro')
            recall = recall_score(labels, predictions, average='micro')
            f1 = f1_score(labels, predictions, average='micro')
            return {'precision': precision, 'recall': recall, 'f1': f1}


        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=val_dataset,
            compute_metrics=compute_metrics,
            callbacks=[loss_logging_callback]
        )

        trainer.train()

        loss_logging_callback.save_logs_to_excel(f"{results_dir}/fold_loss.xlsx")

        results = trainer.evaluate()

        if results['eval_f1'] > best_f1:
            best_f1 = results['eval_f1']
            best_model = model


        predictions = trainer.predict(val_dataset)
        pred_probs = torch.sigmoid(torch.tensor(predictions.predictions)).numpy()
        threshold = 0.5
        binary_predictions = (pred_probs > threshold).astype(int)

        # True labels
        true_labels = predictions.label_ids
        f1 = f1_score(true_labels, binary_predictions, average='micro')

        report_dict = classification_report(true_labels, binary_predictions, output_dict=True, zero_division=0, target_names=['bug report', 'user experience', 'rating', 'feature request'])
        # avg_metrics = report_dict['weighted avg']  # Use 'macro avg' or 'weighted avg' based on your preference
        end_time = time.time()
        # Append the metrics for this fold to the DataFrame
        metrics_df = metrics_df.append({
            ('Fold', ''): fold + 1,
            ('F1-Score', ''): f1,
            ('Train Time', ''): str(end_time - start_time)+" s",
            ('Bug Report', 'P'): report_dict['bug report']['precision'],
            ('Bug Report', 'R'): report_dict['bug report']['recall'],
            ('Bug Report', 'F1'): report_dict['bug report']['f1-score'],
            ('Feature Request', 'P'): report_dict['feature request']['precision'],
            ('Feature Request', 'R'): report_dict['feature request']['recall'],
            ('Feature Request', 'F1'): report_dict['feature request']['f1-score'],
            ('Rating', 'P'): report_dict['rating']['precision'],
            ('Rating', 'R'): report_dict['rating']['recall'],
            ('Rating', 'F1'): report_dict['rating']['f1-score'],
            ('User Experience', 'P'): report_dict['user experience']['precision'],
            ('User Experience', 'R'): report_dict['user experience']['recall'],
            ('User Experience', 'F1'): report_dict['user experience']['f1-score']
        }, ignore_index=True)

    metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in metrics_df.columns])
    metrics_df.to_excel(f"{results_dir}/fold_metrics.xlsx", index=True)

    test_encodings = tokenize_function(X_test_full.tolist())
    test_dataset = MultiLabelDataset(test_encodings, y_test_full)
    test_trainer = Trainer(model=best_model)
    test_predictions = test_trainer.predict(test_dataset)
    test_pred_probs = torch.sigmoid(torch.tensor(test_predictions.predictions)).numpy()
    threshold = 0.5
    test_binary_predictions = (test_pred_probs > threshold).astype(int)

    test_true_labels = test_predictions.label_ids
    test_f1 = f1_score(test_true_labels, test_binary_predictions, average='micro')

    test_report_dict = classification_report(test_true_labels, test_binary_predictions, output_dict=True, zero_division=0, target_names=['bug report', 'user experience', 'rating', 'feature request'])
    # avg_metrics = report_dict['weighted avg']  # Use 'macro avg' or 'weighted avg' based on your preference
    # Append the metrics for this fold to the DataFrame
    test_metrics_df = pd.DataFrame()

    test_metrics_df = test_metrics_df.append({
            ('F1', ''): test_f1,
            ('Bug Report', 'P'): test_report_dict['bug report']['precision'],
            ('Bug Report', 'R'): test_report_dict['bug report']['recall'],
            ('Bug Report', 'F1'): test_report_dict['bug report']['f1-score'],
            ('Feature Request', 'P'): test_report_dict['feature request']['precision'],
            ('Feature Request', 'R'): test_report_dict['feature request']['recall'],
            ('Feature Request', 'F1'): test_report_dict['feature request']['f1-score'],
            ('Rating', 'P'): test_report_dict['rating']['precision'],
            ('Rating', 'R'): test_report_dict['rating']['recall'],
            ('Rating', 'F1'): test_report_dict['rating']['f1-score'],
            ('User Experience', 'P'): test_report_dict['user experience']['precision'],
            ('User Experience', 'R'): test_report_dict['user experience']['recall'],
            ('User Experience', 'F1'): test_report_dict['user experience']['f1-score']
        }, ignore_index=True)

    test_metrics_df.columns = pd.MultiIndex.from_tuples([(c,) if isinstance(c, str) else c for c in test_metrics_df.columns])
    test_metrics_df.to_excel(f"{results_dir}/metrics_results_full_test.xlsx", index=True)

    print(f"Test F1: {test_f1}")

    # Generate and print the classification report
    print(test_report_dict)

    shutil.rmtree(dump_dir)

class MultiLabelDataset(Dataset):

    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float32)  # Ensure float32 for BCEWithLogitsLoss
        return item

    def __len__(self):
        return len(self.labels)
class LossLoggingCallback(TrainerCallback):
    """A custom callback to log training and validation loss."""
    def __init__(self):
        super().__init__()
        self.log_history = []
        self.log_train_loss_history = []

    def on_log(self, args, state, control, logs=None, **kwargs):
        # This method captures both training and evaluation logs, so it's more general than on_epoch_end
        if logs is not None:
            # Capture both training and evaluation steps
            if 'loss' in logs:  # Indicates a training step
                self.log_train_loss_history.append({
                    'epoch': state.epoch,
                    'training_loss': logs.get('loss'),
                })
            elif 'eval_loss' in logs:  # Indicates an evaluation step
                # Make sure to capture the last training loss as well
                last_training_loss = self.log_train_loss_history[-1]['training_loss'] if self.log_train_loss_history else None
                self.log_history.append({
                    'epoch': state.epoch,
                    'training_loss': last_training_loss,  # Include last known training loss for reference
                    'validation_loss': logs.get('eval_loss'),
                    'eval_runtime':logs.get('eval_runtime')
                })

    def save_logs_to_excel(self, file_name):
        """Save the recorded logs to a Excel file."""
        pd.DataFrame(self.log_history).to_excel(file_name, index=False)

__file__ = "/content/drive/MyDrive/FinalProject/Models/ELECTRA/ELECTRA.ipynb"
current_file_path = Path(__file__).parent
path_to_project = current_file_path.parents[1]

directory_path_multi_label = path_to_project / 'Data' / 'Datasets' / 'Multi-label'

files_multi_label = [(file.name, file.stat().st_size)
               for file in directory_path_multi_label.iterdir()
               if file.is_file() and not file.name.startswith('.')]

files_multi_label.sort(key=lambda x: x[1])

In [10]:
print(f"Now doing: {files_multi_label[1][0].split('.')[0]}")
multi_main_model(files_multi_label[1][0].split('.')[0], files_multi_label[1][0].split('.')[1])

Now doing: dataset_gpt_multi_label_4000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.653400,0.582976,0.705948,0.553210,0.620316
2,0.498300,0.425053,0.827263,0.699208,0.757865
3,0.324200,0.297678,0.966551,0.737027,0.836327
4,0.265600,0.266205,0.998819,0.744063,0.852823
5,0.246700,0.262646,0.995316,0.747581,0.853842


<ipython-input-9-6e2d248d9d4c>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.651300,0.607845,0.712150,0.626130,0.666375
2,0.471500,0.372665,0.918675,0.751849,0.826932
3,0.295200,0.272045,0.993583,0.763353,0.863383
4,0.247200,0.257396,0.992585,0.769926,0.867191
5,0.221100,0.247315,0.987578,0.783895,0.874027


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Multi-label/dataset_gpt_multi_label_4000/Dump/res/checkpoint-985 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-9-6e2d248d9d4c>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


<ipython-input-9-6e2d248d9d4c>:170: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_metrics_df = test_metrics_df.append({


Test F1: 0.8806026365348399
{'bug report': {'precision': 0.996309963099631, 'recall': 0.9375, 'f1-score': 0.966010733452594, 'support': 288}, 'user experience': {'precision': 0.9823321554770318, 'recall': 0.9439728353140917, 'f1-score': 0.9627705627705628, 'support': 589}, 'rating': {'precision': 1.0, 'recall': 0.12062256809338522, 'f1-score': 0.2152777777777778, 'support': 257}, 'feature request': {'precision': 0.9842271293375394, 'recall': 0.9285714285714286, 'f1-score': 0.9555895865237367, 'support': 336}, 'micro avg': {'precision': 0.9864978902953586, 'recall': 0.7952380952380952, 'f1-score': 0.8806026365348399, 'support': 1470}, 'macro avg': {'precision': 0.9907173119785506, 'recall': 0.7326667079947264, 'f1-score': 0.7749121651311679, 'support': 1470}, 'weighted avg': {'precision': 0.9885926696639991, 'recall': 0.7952380952380952, 'f1-score': 0.8310798929707979, 'support': 1470}, 'samples avg': {'precision': 0.8677083333333334, 'recall': 0.7525, 'f1-score': 0.7934642857142857, 's

In [11]:
print(f"Now doing: {files_multi_label[2][0].split('.')[0]}")
multi_main_model(files_multi_label[2][0].split('.')[0], files_multi_label[2][0].split('.')[1])
print(f"Now doing: {files_multi_label[3][0].split('.')[0]}")
multi_main_model(files_multi_label[3][0].split('.')[0], files_multi_label[3][0].split('.')[1])

Now doing: dataset_gpt_multi_label_8000
Fold 1/5


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.555500,0.329015,0.927575,0.776381,0.845270
2,0.243800,0.192972,0.993264,0.860799,0.922300
3,0.176900,0.171468,0.996399,0.869780,0.928794
4,0.160900,0.170048,0.997433,0.872474,0.930778
5,0.152100,0.163847,0.996921,0.872474,0.930556


<ipython-input-9-6e2d248d9d4c>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


Fold 2/5


...

Epoch,Training Loss,Validation Loss,Precision,Recall,F1
1,0.361200,0.151996,0.994135,0.890465,0.939449
2,0.133500,0.110774,0.999717,0.927239,0.962115
3,0.100800,0.100266,0.998589,0.929603,0.962862
4,0.079400,0.072384,0.995625,0.956396,0.975616
5,0.054200,0.062248,0.990294,0.964802,0.977382


Checkpoint destination directory /content/drive/MyDrive/FinalProject/Models/ELECTRA/Output/Multi-label/dataset_gpt_multi_label_16000/Dump/res/checkpoint-3940 already exists and is non-empty. Saving will proceed but saved results may be invalid.


<ipython-input-9-6e2d248d9d4c>:133: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append({


<ipython-input-9-6e2d248d9d4c>:170: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_metrics_df = test_metrics_df.append({


Test F1: 0.9786187727175539
{'bug report': {'precision': 0.997737556561086, 'recall': 0.9767441860465116, 'f1-score': 0.987129266927812, 'support': 903}, 'user experience': {'precision': 0.9891572203055693, 'recall': 0.9737991266375546, 'f1-score': 0.9814180929095354, 'support': 2061}, 'rating': {'precision': 0.9827833572453372, 'recall': 0.9170013386880856, 'f1-score': 0.948753462603878, 'support': 747}, 'feature request': {'precision': 0.9940535183349851, 'recall': 0.9794921875, 'f1-score': 0.9867191342843089, 'support': 1024}, 'micro avg': {'precision': 0.9909071227538429, 'recall': 0.9666314677930307, 'f1-score': 0.9786187727175539, 'support': 4735}, 'macro avg': {'precision': 0.9909329131117445, 'recall': 0.961759209718038, 'f1-score': 0.9760049891813836, 'support': 4735}, 'weighted avg': {'precision': 0.9908468881228575, 'recall': 0.9666314677930307, 'f1-score': 0.9785004535574648, 'support': 4735}, 'samples avg': {'precision': 0.8426822916666665, 'recall': 0.831328125, 'f1-score